<a href="https://colab.research.google.com/github/mspendyala/langgraph/blob/main/LangGraphBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langgraph


In [ ]:
pip install graphviz

# Example 1

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Define state structure
class QueryState(TypedDict):
    query: str
    response: str
    next_node: str  # Determines next step

def greeting_agent(state: QueryState) -> QueryState:
    state["response"] = "Hello! How can I assist you?"
    return state

def farewell_agent(state: QueryState) -> QueryState:
    state["response"] = "Goodbye! Have a great day!"
    return state

def query_router(state: QueryState) -> QueryState:
    query = state["query"].lower()

    if "hello" in query:
        return {"query": state["query"], "response": "", "next_node": "greeting"}
    elif "bye" in query:
        return {"query": state["query"], "response": "", "next_node": "farewell"}
    else:
        return {"query": state["query"], "response": "I don't understand.", "next_node": END}


# Create the Graph
workflow = StateGraph(QueryState)

# Add nodes
workflow.add_node("query_router", query_router)
workflow.add_node("greeting", greeting_agent)
workflow.add_node("farewell", farewell_agent)

# Set the entry point
workflow.set_entry_point("query_router")

# Define routing logic
workflow.add_conditional_edges("query_router", lambda state: state["next_node"])

# Set end nodes
workflow.add_edge("greeting", END)
workflow.add_edge("farewell", END)

# Compile the Graph
app = workflow.compile()


# Test cases
queries = ["Hello there!", "Bye for now!", "What is this?"]

for query in queries:
    response = app.invoke({"query": query})
    print(f"Query: {query}\nResponse: {response['response']}\n")



Query: Hello there!
Response: Hello! How can I assist you?

Query: Bye for now!
Response: Goodbye! Have a great day!

Query: What is this?
Response: I don't understand.



# Example 2

In [ ]:
def support_agent(state: QueryState) -> QueryState:
    state["response"] = "Support Agent: How can I help with your issue?"
    return state

def billing_agent(state: QueryState) -> QueryState:
    state["response"] = "Billing Agent: Do you need help with payments?"
    return state

def router(state: QueryState) -> QueryState:
    query = state["query"].lower()

    if "support" in query:
        return {"query": state["query"], "response": "", "next_node": "support"}
    elif "billing" in query:
        return {"query": state["query"], "response": "", "next_node": "billing"}
    else:
        return {"query": state["query"], "response": "No matching department found.", "next_node": END}

workflow = StateGraph(QueryState)

# Add nodes
workflow.add_node("router", router)
workflow.add_node("support", support_agent)
workflow.add_node("billing", billing_agent)

# Set entry point
workflow.set_entry_point("router")

# Define routing logic
workflow.add_conditional_edges("router", lambda state: state["next_node"])

# Set end nodes
workflow.add_edge("support", END)
workflow.add_edge("billing", END)

# Compile
app = workflow.compile()

# Test
queries = ["I need support", "Billing issue", "Random question"]
for query in queries:
    response = app.invoke({"query": query})
    print(f"Query: {query}\nResponse: {response['response']}\n")


Query: I need support
Response: Support Agent: How can I help with your issue?

Query: Billing issue
Response: Billing Agent: Do you need help with payments?

Query: Random question
Response: No matching department found.

